In [ ]:
"""
Ce qu'on veut faire: 
- faire un hungarian avec les L retournés par le module de tracking couplé au  HMM 

Liste des pbs du codes:
- le 4809 génère des nan certainement du au moment où il y'a des detections qui disparaissent ??? 
-du genre on passe de 15 animaux à 14 à investiguer en detail



possible good news to check:
- un switch d'identité serait réglé par le hmm sur le 4809 à la 1761 seconde 
"""



In [ ]:
import numpy as np
from scipy.optimize import linear_sum_assignment
#! pip install mpmath
from mpmath import *
import pandas as pd
import numpy as np 
import datetime as dt
import json 
import copy
import numpy as np 
import cv2

POWERFULNESS=2
Home_folder=  "/home/sophie/HMM-Tracking/Bytetrack"
confidence_on_hmm_choice=1.5
confidence_threshold = 0.2

#en supposant que les observations sont independantes la normalisation à 1 des alpha et beta est acceptable 
#la solution qui suivra sera de choisir l'identité la plus acceptée au niveau de L et de l'affecté à la localisation identifié dans le tracking 

def softmax(x):
    #return x/sum(x)
    #return np.exp(x)/sum(np.exp(x))
    return np.power(x,confidence_on_hmm_choice) / np.sum(np.power(x,confidence_on_hmm_choice), axis=0)

def hungarian_choice(matrice, value_of_confidence_on_track=1):
        row_ind, col_ind = linear_sum_assignment(-matrice)
        for idx,row in enumerate(row_ind):
            matrice[row, col_ind[idx]] = value_of_confidence_on_track
        return matrice

def forward(V=np.array([0,1]), a={"t=1":np.array([[0.5,0.5]]) }, b={"t=0":np.array([0.5]),"t=1":np.array([0.5,0.5])}, initial_distribution=np.array([0.5]), T=1 ): #V=np.array([0,1]), a={"t=1":np.array([[0.5,0.5],[0.5,0.5]]) }, b={"t=0":np.array([0.5,0.5]),"t=1":np.array([0.5,0.5])}, initial_distribution=np.array([0.5,0.5])):
    alpha = {}
    alpha[V[1]] = initial_distribution * b["t="+str(V[1])]
    for t in range(2, V.shape[0]):
        tmp_alpha = alpha[V[t - 1]]
        
        if "t="+str(V[t]) in list(b.keys()) and (b["t="+str(V[t])].max()==b["t="+str(V[t])].min()) and (alpha[V[t-1]].max()== alpha[V[t-1]].min()) :
            #si on a pas d'observation et pas d'observation et un alpha qui est egalitaire, ca ne sert à rien de donner de la force au track auquel letracker est plus confident 
            alpha[V[t]]=np.ones(a["t="+str(V[t])].shape[1]) #ca gère uniquement la première frame, il faudrait trouver un moyen de conserver les valeurs de alpha quand on a rien à la mangeoire
            #print(t,"b=1, alph=1")
        else:
            tmp_b=b["t="+str(V[t])]
            if "t="+str(V[t]) in list(b.keys()) and  (b["t="+str(V[t])].max()==b["t="+str(V[t])].min()) and (alpha[V[t-1]].max()!= alpha[V[t-1]].min()):
                #si on a pas d'observation mais qu'on a une valeur de alpha informative, on lui donne plus de poids pour en donner moins aux confidences du tracker pour eviter que les tracks les plus certains prennent une grande probabilité sur l'identité
                tmp_alpha = np.power(alpha[V[t-1]],POWERFULNESS)  #np.exp(alpha[V[t-1]])
                #print("we leave")
                #print(t,"b=1, alph#1")

            if "t="+str(V[t]) in list(b.keys()) and  b["t="+str(V[t])].max()!=  b["t="+str(V[t])].min() :
                #si on a une observation on ramène le compteur de beta à 0 avec 1 partout à la qui suivais frame
                tmp_alpha= np.ones(alpha[V[t-1]].shape)
                #print(t,"b#1, ")
                
                """alpha[V[t]]=np.zeros(a["t="+str(V[t])].shape[1])
                for j in range(a["t="+str(V[t])].shape[1]):
                    alpha[V[t]][j] = tmp_alpha.dot(a["t="+str(V[t])][:, j]) *(b["t="+str(V[t])][j])
                final_alpha_t=alpha[V[t]]
                print(b["t="+str(V[t])] ,"******", a["t="+str(V[t])] )"""

            alpha[V[t]]=np.zeros(a["t="+str(V[t])].shape[1])
            for j in range(a["t="+str(V[t])].shape[1]):
                alpha[V[t]][j] = tmp_alpha.dot(a["t="+str(V[t])][:, j]) *(b["t="+str(V[t])][j])
            final_alpha_t=alpha[V[t]]

        ##print("*****",V[t], alpha[V[t]])
        if alpha[V[t]].sum()!=0:
            alpha[V[t]]=softmax(alpha[V[t]])
            
        #if t==10198:# or t==281:
        #    print("******************************************************")
        #    print("*****alpha t-1", tmp_alpha,"*****a", a["t="+str(V[t])].shape,"****b", b["t="+str(V[t])] ,"****alpha",alpha[V[t]] ,"\n")
        
            
    return alpha

def backward(V=np.array([0,1]), a={"t=1":np.array([[0.5,0.5]]) }, b={"t=0":np.array([0.5]),"t=1":np.array([0.5,0.5])},  initial_distribution=np.array([0.5]), T=1):
    beta = {}
    # setting beta(T) = 1
    beta[V[-1]] = np.ones((a["t="+str(V[-1])].shape[1]))

    # Loop in backward way from T-1 to
    # Due to python indexing the actual loop will be T-2 to 0
    for t in range(V.shape[0] - 2, -1, -1):
        #tmp_beta=beta[V[t+1]]
        #if "t="+str(V[t]) in list(b.keys()) and (b["t="+str(V[t])].max()==  b["t="+str(V[t])].min()) and  ( beta[V[t+1]].max()== beta[V[t+1]].min()):
        """ ca c'est lorsqu'on a pas d'observation, et que l'on vient de commencer avec beta, on préfère 
        laisser beta t1+1 à 1 et ne pas se laisser influencer par les probabilités de bytetrack
        ??? mais je ne suis pas sure que ca influence par ce que a est normaliser à 1 comme somme de cells par ligne??? à veirifier et tester """
        #  a_temp=np.ones(a["t="+str(V[t+1])].shape)
        #beta[V[t]]=beta[V[t+1]]
        #  beta[V[t]]=np.zeros(a["t="+str(V[t+1])].shape[0])
        #  for j in range(a["t="+str(V[t+1])].shape[0]):
        #    beta[V[t]][j] = (tmp_beta * b["t="+str(V[t + 1])]).dot(a_temp[j, :])
        #else:
        if "t="+str(V[t]) in list(b.keys()):
            tmp_beta=beta[V[t+1]]
            if "t="+str(V[t]) in list(b.keys()) and (b["t="+str(V[t+1])].max()==b["t="+str(V[t+1])].min()) and (beta[V[t+1]].max()== beta[V[t+1]].min()) :
                beta[V[t]]=np.ones(a["t="+str(V[t+1])].shape[0]) #ca gère uniquement la première frame, il faudrait trouver un moyen de conserver les valeurs de alpha quand on a rien à la mangeoire
            else:
                tmp_b=b["t="+str(V[t])]
                if "t="+str(V[t]) in list(b.keys()) and  (b["t="+str(V[t+1])].max()==b["t="+str(V[t+1])].min()) and (beta[V[t+1]].max()!= beta[V[t+1]].min()):
                    tmp_beta = np.power(beta[V[t+1]],POWERFULNESS)  #np.exp(alpha[V[t-1]])

                if "t="+str(V[t]) in list(b.keys()) and  b["t="+str(V[t+1])].max()!=  b["t="+str(V[t+1])].min() :
                    #si on a une observation on ramène le compteur de beta à 0 avec 1 partout à la qui suivais frame
                    ##print("*****we rely on tracking only", b["t="+str(V[t + 1])], tmp_beta * b["t="+str(V[t + 1])] )
                    tmp_beta= np.ones(beta[V[t+1]].shape)

                beta[V[t]]=np.zeros(a["t="+str(V[t+1])].shape[0])
                for j in range(a["t="+str(V[t+1])].shape[0]):
                    beta[V[t]][j] = (tmp_beta * b["t="+str(V[t + 1])]).dot(a["t="+str(V[t+1])][j, :])
            if beta[V[t]].sum()!=0:
                beta[V[t]]=softmax(beta[V[t]])
            ##print("*****",V[t], beta[V[t]])

        #if t==10198:# or t==281:
        #    print("******************************************************")
        #    print("*****beta t+1",beta[V[t]].shape)# tmp_beta,"*****b", a["t="+str(V[t])].shape,"****b", b["t="+str(V[t])] ,"****beta",beta[V[t]] ,"\n")
       
        """if "t="+str(V[t]) in list(b.keys()) :#and  b["t="+str(V[t])].max()!=  b["t="+str(V[t])].min():
            #si on a une observation on ramène le compteur de beta à 0 avec 1 partout à la qui suivais frame
            #tmp_beta= np.ones(beta[V[t+1]].shape)
            tmp_beta=beta[V[t+1]]
            beta[V[t]]=np.zeros(a["t="+str(V[t+1])].shape[0])
            for j in range(a["t="+str(V[t+1])].shape[0]):
                beta[V[t]][j] = (tmp_beta * b["t="+str(V[t + 1])]).dot(a["t="+str(V[t+1])][j, :])

            beta[V[t]]=softmax(beta[V[t]])#sophie mod  beta[V[t]]/beta[V[t]].sum() #"""



    return beta

 
def forward_backward_L(V=np.array([0,1]), a={"t=1":np.array([[0.5,0.25]]) }, b={"t=0":np.array([0.5]),"t=1":np.array([0.5,0.5])}, initial_distribution=np.array([0.5]), T=1 ): #V=np.array([0,1]), a={"t=1":np.array([[0.5,0.5],[0.5,0.5]]) }, b={"t=0":np.array([0.5,0.5]),"t=1":np.array([0.5,0.5])}, initial_distribution=np.array([0.5,0.5])):):
    beta = backward(V,a,b,initial_distribution,T)
    alpha = forward(V,a,b,initial_distribution,T)
    L={}
    for t in V[1:]:
        #if t==10000:
        #    print(alpha[t],"***",beta[t])
        
        L["t="+str(t)]=alpha[t]*beta[t]
        if L["t="+str(t)].sum()!=0:
          L["t="+str(t)]=L["t="+str(t)]/L["t="+str(t)].sum()

        if t==1432:
            print("Final**********",t,L["t="+str(t)],alpha[t], beta[t])
        #print("final t", t)

    return L,beta,alpha

import json
import numpy as np

with open(Home_folder+'/videos/GR77_20200512_111314DBN_resut_with_observations_visits_62.json') as f:
  data = json.load(f)
max_frame=max([int(i) for i in list(data.keys())])

#### on crée la list des identités et ajoutons des noms au format identity'numéros' pour les inconus 
identities=set()
for frame, value in data.items():
  for key in value["observation"].keys():
    # if 'observed' in value.keys():
    identities.add(key)
while len(identities)<15:
  identities.add("identities"+str(len(identities)))



V=[0]
a={}
b={}
for identity in identities:
  b[str(identity)]={}
initial_distribution = np.array([1/len(data["0"]["current"]) for i in data["0"]["current"] ])
for frame_id, value in data.items():
  if  frame_id!="0" and int(frame_id)<max_frame: #int(frame)>100: #(frame!="0" and int(frame)%25==0) or
    a["t="+frame_id]=hungarian_choice(np.array(value["matrice"]))

    for identity in identities:
      b[str(identity)]["t="+frame_id]=np.array([1 for i in value["matrice"][1] ])

    for key in list(value["observation"]):
        b[str(key)]["t="+frame_id]=np.array(value["observation"][key] )
        if b[str(key)]["t="+frame_id].max()>0.1:
            print("observation at frame", frame_id, key)
        
    #if 'observed' in list(value.keys()):
    #  b[str(value["observed"])]["t="+frame_id]=np.array(value["observation"])
    #quand l'animal n'est pas observé ca pourrait être un 1- la d istance à voir mais ca a des conséquence, l'ideal aurait été d'^tre sure quant à celui qui est à la mangeoire
    V.append(int(frame_id))


V=V[:-1]
V=np.array(V)
T=len(V)-2
L={}
Beta={}
Alpha={}


for identity in list(identities) [:15]:
    if True:# identity=='4812.0':
        L[identity], Beta[identity], Alpha[identity]= forward_backward_L(V=V,  a=a, b=b[str(identity)], initial_distribution=initial_distribution, T=V[-1] )
        print(identity, "process finished")


####This part for matching  atq identities to objects ###########
import pandas as pd
plot_alpha={}
plot_beta={}
plot_L=pd.DataFrame(columns=[i for i in range(16)])
#print(plot)
#on va observer 4808

In [ ]:
import pandas as pd
#!pip install plotly
import plotly.express as px

plot_alpha={}
plot_beta={}
plot_L={}
for identity in identities:
    plot_L[identity]=pd.DataFrame(columns=[i+1 for i in range(15)])
    plot_alpha[identity]=pd.DataFrame(columns=[i+1 for i in range(15)])
    plot_beta[identity]=pd.DataFrame(columns=[i+1 for i in range(15)])

for t in V[1:] :
    #print(t)
    for identity in identities:
        if identity=='4812.0':
            plot_L[identity].loc[t]= [None for i in range(15)]
            plot_alpha[identity].loc[t]= [None for i in range(15)]
            plot_beta[identity].loc[t]= [None for i in range(15)]
            for idx, track in enumerate(data[str(t)]["current"]) :
                plot_L[identity].loc[t, track["track_id"]]=L[identity]["t="+str(t)][idx]
                plot_alpha[identity].loc[t, track["track_id"]]= Alpha[identity][t][idx]
                #print(Alpha[identity][t][idx])
                plot_beta[identity].loc[t, track["track_id"]]=Beta[identity][t][idx]
               
                
    


In [ ]:
pd.set_option('display.max_rows', None)
plot_L["4812.0"]


In [ ]:
for identity in identities:
    if identity in identities:#["4808.0"]:
        plot_L[identity]["index"]=plot_L[identity].index
        plot_alpha[identity]["index"]=plot_alpha[identity].index
        plot_beta[identity]["index"]=plot_beta[identity].index
        
        fig_alpha = px.line(plot_alpha[identity], x="index", y=plot_alpha[identity].columns, title="alpha for "+str(identity))
        fig_beta = px.line(plot_beta[identity], x="index", y=plot_alpha[identity].columns, title="beta for "+str(identity))
        fig_L = px.line(plot_L[identity], x="index", y=plot_alpha[identity].columns, title="L for "+str(identity))
        """fig_L = px.line(plot_L[identity], x="index", y=plot_L[identity].columns, title="L for "+str(identity))
        fig_alpha = px.line(plot_alpha[identity], x="index", y=plot_alpha[identity].columns, title="alpha for "+str(identity))
        fig_beta = px.line(plot_beta[identity], x="index", y=plot_beta[identity].columns, title="beta for "+str(identity))
        """
        fig_L.show()
        fig_alpha.show()
        fig_beta.show()


In [ ]:
print(list(identities))
for t in V[1:] : 
        matrice=np.zeros((len(L[list(identities)[0]]["t="+str(t)]),len(list(identities))))
        for idx,identity in enumerate(list(identities)[:15]):
            matrice[:,idx]= L[identity]["t="+str(t)]
        #hungarian fin the correspondance with the minimal cost, since we want to maximize the  sum  of probabilities,  we will use -probability  
        """matrice_df={}
        for identity in identities:
                matrice_df[identity]= L[identity]["t="+str(t)]
        matrice_df=pd.DataFrame(matrice_df)"""

        #######Hungarian version who seems to be ok   
        #try:
        #    row_ind, col_ind = linear_sum_assignment(-matrice) #since the function is looking for the assignement minimizing the sum, we put the opposite of the propabiliy in cells 
        #except:
        #    print("xeption on this matrix")#,t,list(identities)[3], L[list(identities)[3]]["t="+str(t)], matrice)            
        #for idx, row in enumerate(row_ind):
        #  data[str(t)]["current"][idx]["atq"] = list(identities)[col_ind[idx]]
            
        
        ##########version with the maximum one feeting 
        for idx, track in enumerate(data[str(t)]["current"]):
            track["atq"] = "None"
            identity_with_max_val= np.argmax(matrice[idx])
            #print(identity_with_max_val, list(identities)[identity_with_max_val])
            if  L[list(identities)[identity_with_max_val]]["t="+str(t)][idx]>0.2:
                data[str(t)]["current"][idx]["atq"] = list(identities)[identity_with_max_val]


In [ ]:
def get_track_from_id_and_time(track_id,t):
     for idx, track in enumerate(data[str(t)]["current"]):
            if track["track_id"]==track_id:
                return track
"""def get_previous_track_from_id(track_id,t,tmp=1):
    while t-tmp>0:
        previous_track = get_track_from_id_and_time(track_id,t-tmp)
        tmp+=1
        if previous_track!=None:
            return previous_track
    
def get_future_track_from_id(track_id,t,tmp=1):
    while t+tmp>0:
        future_track = get_track_from_id_and_time(track_id,t+tmp)
        tmp+=1
        if future_track!=None:
            return future_track
    """
    
    
def smooting_from_past(data, gap=750):
    for t in V[1:] : 
         for idx, track in enumerate(data[str(t)]["current"]):
                if track["atq"]=="None":
                    track_id=track["track_id"]
                    track["atq_from_previous"]="None"
                    if t>gap:
                        track_previous = get_track_from_id_and_time(track_id,t-1)
                        track_far_previous = get_track_from_id_and_time(track_id,t-gap)
                        if track_previous!=None and track_far_previous!=None:
                            if track_previous["atq_from_previous"]!="None" and track_far_previous["atq_from_previous"]!="None":
                                if track_previous["atq_from_previous"]== track_far_previous["atq_from_previous"]:
                                    print(t)
                                    print(track)
                                    data[str(t)]["current"][idx]["atq_from_previous"]= track_previous["atq_from_previous"]
                                    #print(track)
                        #except(e):
                    #    print("an exception occur this is its description:",e)
                else:
                    data[str(t)]["current"][idx]["atq_from_previous"]= track["atq"]


def smooting_from_future(data, gap=750):
    for t in reversed(V[1:]) : 
         for idx, track in enumerate(data[str(t)]["current"]):
                if track["atq"]=="None":
                    track_id=track["track_id"]
                    track["atq_from_future"]="None"
                    if V[-1]-t>gap:
                        track_future = get_track_from_id_and_time(track_id,t+1)
                        track_far_future = get_track_from_id_and_time(track_id,t+gap)

                        if track_future!=None and track_far_future!=None:

                            if track_future["atq_from_future"]!="None" and track_far_future["atq_from_future"]!="None":

                                if track_future["atq_from_future"]== track_far_future["atq_from_future"]:
                                    print(t)

                                    data[str(t)]["current"][idx]["atq_from_future"]= track_future["atq_from_future"]
                                    print( data[str(t)]["current"][idx])
                                    #print(track)
                        #except(e):
                    #    print("an exception occur this is its description:",e)
                else:
                    data[str(t)]["current"][idx]["atq_from_future"]= track["atq"]

smooting_from_future(data,gap=float('inf')) #=1000)#   4à secondes de gap
smooting_from_past(data,gap=float('inf')) # 1000)#

In [ ]:
##############writting on the video#########################

  
#print(tracks_with_atq)
video_path="/home/sophie/HMM-Tracking/Bytetrack/videos/GR77_20200512_111314.mp4"
cap = cv2.VideoCapture(video_path)
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)  # float
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)  # float
fps = cap.get(cv2.CAP_PROP_FPS)
save_path= "/home/sophie/HMM-Tracking/Bytetrack/videos/GR77_20200512_111314_with_atq24.mp4"
vid_writer = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (int(width), int(height)))     




# Center coordinates
center_coordinates = (625, 70)
 
# Radius of circle
radius = 2
  
# Blue color in BGR
color = (255, 0, 0)
  
# Line thickness of 2 px
thickness = 2
  
# Using cv2.circle() method
# Draw a circle with blue line borders of thickness of 2 px
#frame = cv2.circle(frame, center_coordinates, radius, color, thickness)
  
  
#cv2.imwrite("image_.jpg", frame) 

print("********************** we start writting in the video")
track_file="/home/sophie/HMM-Tracking/Bytetrack/videos/GR77_20200512_111314tracking_resut.json"
with open(track_file) as f:
        tracks = json.load(f) 
        
"""print(tracks.keys())    
exit(0)    """
ret_val, frame = cap.read()
frame_id=1
tracking_result={}
while ret_val and frame_id!=V[-1]: 
    dct={}
    ret_val, frame = cap.read()
    frame = cv2.circle(frame, center_coordinates, radius, color, thickness)
    if str(frame_id) in data.keys():
      if (frame!="0" ):
        cv2.putText(frame, str(frame_id),(90+580, 20),0, 5e-3 * 200, (0,255,0),2)
        for track in data[str(frame_id)]["current"]:
            track_id=track["track_id"] 
            tlwh = track["location"]
            print(track)
            atq= track["atq"] 
            if atq=="None":
                if track["atq_from_previous"]!="None":
                    atq= track["atq_from_previous"]+'fp'
                elif track["atq_from_future"]!="None":
                    atq= track["atq_from_future"]+'ff'
                    
            tid= str(track_id)+", atq:"+str(atq)
            print(tid)
            if atq!="None":
                dct[atq]=(int(tlwh[0]), int(tlwh[1]), int(tlwh[2]), int(tlwh[3]) )
            cv2.rectangle(frame, (int(tlwh[0]), int(tlwh[1])), (int(tlwh[0])+int(tlwh[2]), int(tlwh[1])+int(tlwh[3])) ,(255,255,255), 2)
            cv2.rectangle(frame, (580, 20), (90+580, 115+20) ,(255,255,255), 2)

            cv2.putText(frame, str(tid),(int(tlwh[0]), int(tlwh[1])),0, 5e-3 * 200, (0,255,0),2)
            
        tracking_result[frame_id]=dct
        vid_writer.write(frame)
    print("\n", "\n")
    frame_id=frame_id+1
with open(save_path.split('.mp4')[0]+'tracking_with_HMM_resut.json', 'w') as outfile:
    json.dump(tracking_result, outfile)
vid_writer.release()
#plutôt la surface d'intersection des rectangles plutôt que la distance eucledienne 
